In [1]:
%load_ext autoreload
%autoreload 2
import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import mplhep as hep
import hist

import sys
sys.path.append("/workspaces/cmssw/analyzer")
from tools import *

In [2]:
reader = DumperReader("../../runs/AlessandroElectronSamples/histo.root") #("../runs_results/histo.root:ticlDumper")

In [6]:
reader.supercluster_df

ts_id
eventInternal supercls_id ts_in_supercls_id       
0             0           0                   2337
                          1                   2134
                          2                   2294
                          3                   2524
                          4                   2127
...                                            ...
9             293         1                   2007
              294         0                   2595
                          1                   2418
              295         0                   3000
                          1                   2646

[5993 rows x 1 columns]

In [7]:
reader.supercluster_joinTracksters_df

ts_id  raw_energy  \
eventInternal supercls_id ts_in_supercls_id                      
0             0           0                   2337   56.334934   
                          1                   2134   49.254459   
                          2                   2294   50.082699   
                          3                   2524    3.359977   
                          4                   2127    3.850118   
...                                            ...         ...   
9             293         1                   2007    0.994052   
              294         0                   2595    0.859682   
                          1                   2418    1.318508   
              295         0                   3000    1.203344   
                          1                   2646    5.756074   

                                             regressed_energy  
eventInternal supercls_id ts_in_supercls_id                    
0             0           0                         50.047886  
                          1                         51.563450  
                          2                         55.172504  
                          3                          6.817931  
                          4                          8.440736  
...                                                       ...  
9             293         1                          0.000000  
              294         0                          0.000000  
                          1                          4.570274  
              295         0                          5.071675  
                          1                         10.941783  

[5993 rows x 3 columns]

In [20]:
reader.supercluster_merged_properties_all

raw_energy_supercls  regressed_energy_supercls
eventInternal supercls_id                                                
0             0                     169.952988                 186.268738
              1                     139.506744                 158.281235
              2                      69.507591                  79.805992
              3                      13.705381                  29.611601
              4                      21.025515                  27.357998
...                                        ...                        ...
9             3463                    1.613498                   5.455769
              3464                    1.433856                   4.327745
              3465                    1.064146                   4.103632
              3466                    1.036201                   4.255516
              3467                    0.935577                   0.000000

[31937 rows x 2 columns]

In [39]:
reader.supercluster_df.loc[0].query("ts_id == 500 or ts_id == 586 or ts_id == 774 or ts_id == 732")

ts_id
supercls_id ts_in_supercls_id       
1           0                    774
            1                    732
2           0                    586
54          1                    500

In [32]:
assocs_zip_simToReco(reader.associations)[0][0]

<Array [{simts_id: 0, ts_id: 500, ...}, ...] type='3141 * {simts_id: int64,...'>

In [46]:
reader.file["simtrackstersSC"].show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
event                | uint32_t                 | AsDtype('>u4')
NTracksters          | uint32_t                 | AsDtype('>u4')
time                 | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
timeError            | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
regressed_energy     | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
regressed_pt         | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
raw_energy           | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
raw_em_energy        | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
raw_pt               | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
raw_em_pt            | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
barycenter_x         | std::vector<float>       | AsJagged(A

In [12]:
assocs_simToReco_largestScore = assocs_bestScore(assocs_zip_simToReco(reader.associations))


In [15]:
df_mainAssoc:pd.DataFrame = (ak.to_dataframe(assocs_simToReco_largestScore[["ts_id"]], 
                            levelname=lambda x : {0:"eventInternal", 1:"genparticle_id"}[x])
        .reset_index("genparticle_id")
        .set_index("ts_id", append=True)
    )
df_mainAssoc

genparticle_id
eventInternal ts_id                
0             500                 0
              2134                1
1             794                 0
              2525                1
2             652                 0
              2224                1
3             785                 0
              2647                1
4             779                 0
              2308                1
5             737                 0
              2243                1
6             968                 0
              2527                1
7             801                 0
              2529                1
8             1183                0
              3225                1
9             842                 0
              2829                1

In [23]:
merged_tsId = pd.merge(df_mainAssoc, reader.supercluster_df, how="left", left_index=True, right_on=["eventInternal", "ts_id"]).reset_index("ts_in_supercls_id", drop=True)
merged_tsId

genparticle_id  ts_id
eventInternal supercls_id                       
0             54                        0    500
              0                         1   2134
1             2                         0    794
              295                       1   2525
2             1                         0    652
              2                         1   2224
3             3                         0    785
              2                         1   2647
4             1                         0    779
              2                         1   2308
5             2                         0    737
              0                         1   2243
6             1                         0    968
              2                         1   2527
7             295                       0    801
              1                         1   2529
8             3                         0   1183
              9                         1   3225
9             2                         0    842
              1                         1   2829

In [25]:
pd.merge(merged_tsId, reader.supercluster_merged_properties_all, left_index=True, right_index=True)

genparticle_id  ts_id  raw_energy_supercls  \
eventInternal supercls_id                                               
0             54                        0    500            72.663040   
              0                         1   2134           169.952988   
1             2                         0    794           195.886124   
              295                       1   2525             4.506697   
2             1                         0    652           267.032257   
              2                         1   2224           558.274780   
3             3                         0    785           270.012756   
              2                         1   2647           250.883453   
4             1                         0    779           374.847046   
              2                         1   2308           179.098740   
5             2                         0    737           183.937622   
              0                         1   2243           330.052643   
6             1                         0    968           369.903778   
              2                         1   2527           208.263611   
7             295                       0    801             5.807618   
              1                         1   2529           129.825623   
8             3                         0   1183            39.468246   
              9                         1   3225            31.864899   
9             2                         0    842           499.658813   
              1                         1   2829           255.378616   

                           regressed_energy_supercls  
eventInternal supercls_id                             
0             54                           82.985603  
              0                           186.268738  
1             2                           218.991425  
              295                           8.489294  
2             1                           290.506714  
              2                           596.428650  
3             3                           310.952667  
              2                           275.164398  
4             1                           404.542450  
              2                           202.148087  
5             2                           197.532333  
              0                           363.829590  
6             1                           413.804749  
              2                           236.317444  
7             295                          11.614075  
              1                           140.398880  
8             3                            74.036278  
              9                            57.363464  
9             2                           541.014771  
              1                           317.974792

In [29]:
reader.getCPToSuperclusterProperties()

supercls_id  seed_ts_id  raw_energy_supercls  \
eventInternal genparticle_id                                                 
0             0                        54         500            72.663040   
              1                         0        2134           169.952988   
1             0                         2         794           195.886124   
              1                      2261        2525           194.904221   
2             0                         1         652           267.032257   
              1                         2        2224           558.274780   
3             0                         3         785           270.012756   
              1                         2        2647           250.883453   
4             0                         1         779           374.847046   
              1                         2        2308           179.098740   
5             0                         2         737           183.937622   
              1                         0        2243           330.052643   
6             0                         1         968           369.903778   
              1                         2        2527           208.263611   
7             0                       836         801           162.093094   
              1                         1        2529           129.825623   
8             0                         3        1183            39.468246   
              1                         9        3225            31.864899   
9             0                         2         842           499.658813   
              1                         1        2829           255.378616   

                              regressed_energy_supercls  
eventInternal genparticle_id                             
0             0                               82.985603  
              1                              186.268738  
1             0                              218.991425  
              1                              205.276733  
2             0                              290.506714  
              1                              596.428650  
3             0                              310.952667  
              1                              275.164398  
4             0                              404.542450  
              1                              202.148087  
5             0                              197.532333  
              1                              363.829590  
6             0                              413.804749  
              1                              236.317444  
7             0                              172.884918  
              1                              140.398880  
8             0                               74.036278  
              1                               57.363464  
9             0                              541.014771  
              1                              317.974792